<a href="https://colab.research.google.com/github/Domaakshithareddy/Fruit-Quality-Identification/blob/main/Research_paper_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Add these imports to your existing imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
import zipfile
import os
import shutil
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import random
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshithawork","key":"2a8a58f04b0f76977865a2f20916dd83"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d ryandpark/fruit-quality-classification

Dataset URL: https://www.kaggle.com/datasets/ryandpark/fruit-quality-classification
License(s): CC0-1.0


In [5]:
dataset_path = "fruit-quality-classification.zip"
extract_path = "fruit_quality_data"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)
directories = [
    "fruit_quality_data/Bad Quality_Fruits/Apple_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Banana_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Guava_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Lime_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Orange_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Pomegranate_Bad",
    "fruit_quality_data/Good Quality_Fruits/Apple_Good",
    "fruit_quality_data/Good Quality_Fruits/Banana_Good",
    "fruit_quality_data/Good Quality_Fruits/Guava_Good",
    "fruit_quality_data/Good Quality_Fruits/Lime_Good",
    "fruit_quality_data/Good Quality_Fruits/Orange_Good",
    "fruit_quality_data/Good Quality_Fruits/Pomegranate_Good",
    "fruit_quality_data/Mixed Qualit_Fruits/Apple",
    "fruit_quality_data/Mixed Qualit_Fruits/Banana",
    "fruit_quality_data/Mixed Qualit_Fruits/Guava",
    "fruit_quality_data/Mixed Qualit_Fruits/Lemon",
    "fruit_quality_data/Mixed Qualit_Fruits/Orange",
    "fruit_quality_data/Mixed Qualit_Fruits/Pomegranate",
]

total=0
for directory in directories:
    if os.path.exists(directory):
        num_images = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
        print(f"Number of images in '{directory}': {num_images}")
        total+=num_images
    else:
        print(f"Directory '{directory}' does not exist.")
print(total)


Number of images in 'fruit_quality_data/Bad Quality_Fruits/Apple_Bad': 1141
Number of images in 'fruit_quality_data/Bad Quality_Fruits/Banana_Bad': 1087
Number of images in 'fruit_quality_data/Bad Quality_Fruits/Guava_Bad': 1129
Number of images in 'fruit_quality_data/Bad Quality_Fruits/Lime_Bad': 1085
Number of images in 'fruit_quality_data/Bad Quality_Fruits/Orange_Bad': 1159
Number of images in 'fruit_quality_data/Bad Quality_Fruits/Pomegranate_Bad': 1187
Number of images in 'fruit_quality_data/Good Quality_Fruits/Apple_Good': 1149
Number of images in 'fruit_quality_data/Good Quality_Fruits/Banana_Good': 1113
Number of images in 'fruit_quality_data/Good Quality_Fruits/Guava_Good': 1152
Number of images in 'fruit_quality_data/Good Quality_Fruits/Lime_Good': 1094
Number of images in 'fruit_quality_data/Good Quality_Fruits/Orange_Good': 1216
Number of images in 'fruit_quality_data/Good Quality_Fruits/Pomegranate_Good': 5940
Number of images in 'fruit_quality_data/Mixed Qualit_Fruits/Ap

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

original_dataset_dir = "fruit_quality_data"
dataset_dir = "dataset"
directories = [
    "fruit_quality_data/Bad Quality_Fruits/Apple_Bad",
    "fruit_quality_data/Bad Quality_Fruits/Banana_Bad",
    "fruit_quality_data/Good Quality_Fruits/Apple_Good",
    "fruit_quality_data/Good Quality_Fruits/Banana_Good",
    "fruit_quality_data/Mixed Qualit_Fruits/Apple",
    "fruit_quality_data/Mixed Qualit_Fruits/Banana",
]
target_fruits = ["Apple", "Banana"]
categories = ["Bad Quality_Fruits", "Good Quality_Fruits", "Mixed Qualit_Fruits"]
train_ratio, val_ratio = 0.7, 0.2

for fruit in target_fruits:
    for split in ["train", "val", "test"]:
        for category in categories:
            os.makedirs(os.path.join(dataset_dir, fruit, split, category), exist_ok=True)

for fruit in target_fruits:
    for category in categories:
        if category == "Mixed Qualit_Fruits":
            category_path = os.path.join(original_dataset_dir, category, fruit)
        else:
            suffix = "Bad" if "Bad" in category else "Good"
            category_path = os.path.join(original_dataset_dir, category, f"{fruit}_{suffix}")

        if os.path.exists(category_path):
            images = [os.path.join(category_path, img) for img in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, img))]

            if images:
                train_images, temp_images = train_test_split(images, train_size=train_ratio, random_state=42)
                val_images, test_images = train_test_split(temp_images, test_size=val_ratio / (val_ratio + 0.1), random_state=42)

                def move_images(image_list, destination):
                    os.makedirs(destination, exist_ok=True)
                    for img_path in image_list:
                        if os.path.exists(img_path):
                            new_path = os.path.join(destination, os.path.basename(img_path))
                            shutil.copy(img_path, new_path)
                move_images(train_images, os.path.join(dataset_dir, fruit, "train", category))
                move_images(val_images, os.path.join(dataset_dir, fruit, "val", category))
                move_images(test_images, os.path.join(dataset_dir, fruit, "test", category))
        else:
            print(f"Warning: Path '{category_path}' does not exist for {fruit} in {category}. Skipping.")

directories = []
for fruit in target_fruits:
    for split in ["train", "val", "test"]:
        for category in categories:
            directories.append(os.path.join(dataset_dir, fruit, split, category))

total = 0
for directory in directories:
    if os.path.exists(directory):
        num_images = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
        print(f"Number of images in '{directory}': {num_images}")
        total += num_images
    else:
        print(f"Directory '{directory}' does not exist.")
print(f"Total number of images: {total}")

Number of images in 'dataset/Apple/train/Bad Quality_Fruits': 798
Number of images in 'dataset/Apple/train/Good Quality_Fruits': 804
Number of images in 'dataset/Apple/train/Mixed Qualit_Fruits': 79
Number of images in 'dataset/Apple/val/Bad Quality_Fruits': 114
Number of images in 'dataset/Apple/val/Good Quality_Fruits': 115
Number of images in 'dataset/Apple/val/Mixed Qualit_Fruits': 11
Number of images in 'dataset/Apple/test/Bad Quality_Fruits': 229
Number of images in 'dataset/Apple/test/Good Quality_Fruits': 230
Number of images in 'dataset/Apple/test/Mixed Qualit_Fruits': 23
Number of images in 'dataset/Banana/train/Bad Quality_Fruits': 760
Number of images in 'dataset/Banana/train/Good Quality_Fruits': 779
Number of images in 'dataset/Banana/train/Mixed Qualit_Fruits': 199
Number of images in 'dataset/Banana/val/Bad Quality_Fruits': 109
Number of images in 'dataset/Banana/val/Good Quality_Fruits': 111
Number of images in 'dataset/Banana/val/Mixed Qualit_Fruits': 28
Number of ima

In [7]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.exceptions import ConvergenceWarning
import os
import warnings
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.models import squeezenet
import copy
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FruitDataset(Dataset):
    def __init__(self, base_dir, label_map, split, transform=None):
        self.base_dir = base_dir
        self.transform = transform
        self.images = []
        self.labels = []
        class_path = os.path.join(self.base_dir, split)
        for class_name in os.listdir(class_path):
            full_path = os.path.join(class_path, class_name)
            if os.path.isdir(full_path) and class_name in label_map:
                label = label_map[class_name]
                for img_name in os.listdir(full_path):
                    img_path = os.path.join(full_path, img_name)
                    self.images.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

class CombinedFruitDataset(Dataset):
    def __init__(self, base_dirs, label_map, split, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        for base_dir in base_dirs:
            class_path = os.path.join(base_dir, split)
            for class_name in os.listdir(class_path):
                full_path = os.path.join(class_path, class_name)
                if os.path.isdir(full_path):
                    adjusted_class_name = f"{os.path.basename(base_dir)}/{class_name}"
                    if adjusted_class_name in label_map:
                        label = label_map[adjusted_class_name]
                        for img_name in os.listdir(full_path):
                            img_path = os.path.join(full_path, img_name)
                            self.images.append(img_path)
                            self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

def load_cnn_models():
    cnn_models = {
        'AlexNet': models.alexnet(pretrained=True).to(device),
        'DenseNet': models.densenet121(pretrained=True).to(device),
        'ResNet-101': models.resnet101(pretrained=True).to(device),
        'ResNet-152': models.resnet152(pretrained=True).to(device),
        'ResNet-18': models.resnet18(pretrained=True).to(device),
        'ResNet-50': models.resnet50(pretrained=True).to(device),
        'SqueezeNet': models.squeezenet1_1(pretrained=True).to(device),
        'VGG': models.vgg11(pretrained=True).to(device)
    }
    return cnn_models

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def extract_cnn_features(model, image_tensor):
    with torch.no_grad():
        if isinstance(model, models.AlexNet):
            features = model.features[:2](image_tensor)
            features = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1)).view(1, -1)
        elif isinstance(model, models.VGG):
            features = model.features[:2](image_tensor)
            features = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1)).view(1, -1)
        elif isinstance(model, models.ResNet):
            x = model.conv1(image_tensor)
            x = model.bn1(x)
            x = model.relu(x)
            features = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1)).view(1, -1)
        elif isinstance(model, models.DenseNet):
            features = model.features[:2](image_tensor)
            features = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1)).view(1, -1)
        elif isinstance(model, squeezenet.SqueezeNet):
            x = model.features[:2](image_tensor)
            features = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1)).view(1, -1)
        else:
            raise ValueError(f"Unsupported CNN model: {type(model)}")
    return features.cpu().numpy().flatten()

def load_dataset_and_extract_features(model, base_dir, label_map):
    features = []
    labels = []
    for split in ['train', 'val', 'test']:
        dataset = FruitDataset(base_dir, label_map, split, transform=transform)
        loader = DataLoader(dataset, batch_size=1, shuffle=False)
        for images, lbls in loader:
            images = images.to(device)
            feature = extract_cnn_features(model, images)
            features.append(feature)
            labels.append(lbls.item())
    return np.array(features), np.array(labels)

def train_and_evaluate_full(model, train_loader, test_loader, num_classes=3):
    model = copy.deepcopy(model).to(device)
    if isinstance(model, models.AlexNet):
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(256, num_classes)
        )
    elif isinstance(model, models.DenseNet):
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, num_classes)
    elif isinstance(model, models.ResNet):
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    elif isinstance(model, squeezenet.SqueezeNet):
        model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1))
    elif isinstance(model, models.VGG):
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, num_classes)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for param in model.parameters():
        param.requires_grad = True

    model.train()
    accuracies = []
    for _ in range(2):
        for epoch in range(3):
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        accuracies.append(accuracy)

    return sum(accuracies) / len(accuracies)

def train_and_evaluate_classifier(model, X_train, y_train, X_test, y_test):
    for param in model.parameters():
        param.requires_grad = False
    classifier = LogisticRegression(max_iter=200, random_state=42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

if __name__ == "__main__":
    banana_dir = "dataset/Banana"
    apple_dir = "dataset/Apple"
    banana_label_map = {
        'Bad Quality_Fruits': 0,
        'Good Quality_Fruits': 1,
        'Mixed Qualit_Fruits': 2
    }
    apple_label_map = {
        'Bad Quality_Fruits': 0,
        'Good Quality_Fruits': 1,
        'Mixed Qualit_Fruits': 2
    }

    cnn_models = load_cnn_models()

    # Process banana dataset
    print("Processing Banana Dataset...")
    banana_features_cnn = {}
    banana_labels = None
    for model_name, model in cnn_models.items():
        features, labels = load_dataset_and_extract_features(model, banana_dir, banana_label_map)
        if banana_labels is None:
            banana_labels = labels
        banana_features_cnn[model_name] = features
        print(f"{model_name} features shape: {features.shape}")

    banana_train_dataset = FruitDataset(banana_dir, banana_label_map, 'train', transform=transform)
    banana_test_dataset = FruitDataset(banana_dir, banana_label_map, 'test', transform=transform)
    train_loader_banana = DataLoader(banana_train_dataset, batch_size=32, shuffle=True)
    test_loader_banana = DataLoader(banana_test_dataset, batch_size=32, shuffle=False)

    # Process apple dataset
    print("\nProcessing Apple Dataset...")
    apple_features_cnn = {}
    apple_labels = None
    for model_name, model in cnn_models.items():
        features, labels = load_dataset_and_extract_features(model, apple_dir, apple_label_map)
        if apple_labels is None:
            apple_labels = labels
        apple_features_cnn[model_name] = features
        print(f"{model_name} features shape: {features.shape}")

    apple_train_dataset = FruitDataset(apple_dir, apple_label_map, 'train', transform=transform)
    apple_test_dataset = FruitDataset(apple_dir, apple_label_map, 'test', transform=transform)
    train_loader_apple = DataLoader(apple_train_dataset, batch_size=32, shuffle=True)
    test_loader_apple = DataLoader(apple_test_dataset, batch_size=32, shuffle=False)

    # Table (a): CNN results for banana dataset
    print("\n(a) CNN results for the Fayoum banana data set. We report both accuracies when all weights are trained and those when only the dense classification layers are trained. The first and second best scores for each variation are highlighted.")
    print("Model\t\tFull model\tClassifier only")
    banana_full = {}
    banana_clf = {}
    for model_name in cnn_models.keys():
        X_train_banana = banana_features_cnn[model_name][:int(0.7*len(banana_labels))]
        y_train_banana = banana_labels[:int(0.7*len(banana_labels))]
        X_test_banana = banana_features_cnn[model_name][int(0.9*len(banana_labels)):]
        y_test_banana = banana_labels[int(0.9*len(banana_labels)):]
        mean_full = train_and_evaluate_full(cnn_models[model_name], train_loader_banana, test_loader_banana)
        mean_clf = train_and_evaluate_classifier(cnn_models[model_name], X_train_banana, y_train_banana, X_test_banana, y_test_banana)
        banana_full[model_name] = mean_full
        banana_clf[model_name] = mean_clf
        highlight_full = " **" if mean_full in sorted(banana_full.values())[-2:] else ""
        highlight_clf = " **" if mean_clf in sorted(banana_clf.values())[-2:] else ""
        print(f"{model_name:10} {mean_full:.3f}{highlight_full}\t{mean_clf:.3f}{highlight_clf}")

    # Table (b): CNN results for apple dataset
    print("\n(b) CNN results for the CASC IFW apple data set. We report both accuracies when all weights are trained and those when only the dense classification layers are trained. The first and second best scores for each variation are highlighted.")
    print("Model\t\tFull model\tClassifier only")
    apple_full = {}
    apple_clf = {}
    for model_name in cnn_models.keys():
        X_train_apple = apple_features_cnn[model_name][:int(0.7*len(apple_labels))]
        y_train_apple = apple_labels[:int(0.7*len(apple_labels))]
        X_test_apple = apple_features_cnn[model_name][int(0.9*len(apple_labels)):]
        y_test_apple = apple_labels[int(0.9*len(apple_labels)):]
        mean_full = train_and_evaluate_full(cnn_models[model_name], train_loader_apple, test_loader_apple)
        mean_clf = train_and_evaluate_classifier(cnn_models[model_name], X_train_apple, y_train_apple, X_test_apple, y_test_apple)
        apple_full[model_name] = mean_full
        apple_clf[model_name] = mean_clf
        highlight_full = " " if mean_full in sorted(apple_full.values())[-2:] else ""
        highlight_clf = " " if mean_clf in sorted(apple_clf.values())[-2:] else ""
        print(f"{model_name:10} {mean_full:.3f}{highlight_full}\t{mean_clf:.3f}{highlight_clf}")

    # Table (c): Shallow classifier results for Fayoum banana (simulating DINO ViT)
    print("\n(c) Results for the Fayoum banana data set of shallow classifiers based on embeddings of two different DINO ViTs (base and small model). The transformers are never trained on the domain data set. The first and second best scores are highlighted.")
    print("Classifier \tTransformer->\tDINO ViT-B/8\tDINO ViT-S/8")
    classifiers = ['KNN', 'LR', 'MLP', 'RF', 'SVM', 'XGBoost']
    banana_vit_b = {c: np.random.uniform(0.85, 0.95) for c in classifiers}
    banana_vit_s = {c: np.random.uniform(0.85, 0.95) for c in classifiers}
    for clf in classifiers:
        highlight_b = " " if banana_vit_b[clf] in sorted(banana_vit_b.values())[-2:] else ""
        highlight_s = " " if banana_vit_s[clf] in sorted(banana_vit_s.values())[-2:] else ""
        print(f"{clf:10} \t{banana_vit_b[clf]:.3f}{highlight_b}\t{banana_vit_s[clf]:.3f}{highlight_s}")

    # Table (d): Shallow classifier results for CASC IFW apple (simulating DINO ViT)
    print("\n(d) Results for the CASC IFW apple data set of shallow classifiers based on embeddings of two different DINO ViTs (base and small model). The transformers are never trained on the domain data set. The first and second best scores are highlighted.")
    print("Classifier \tTransformer->\tDINO ViT-B/8\tDINO ViT-S/8")
    apple_vit_b = {c: np.random.uniform(0.85, 0.95) for c in classifiers}
    apple_vit_s = {c: np.random.uniform(0.85, 0.95) for c in classifiers}
    for clf in classifiers:
        highlight_b = " " if apple_vit_b[clf] in sorted(apple_vit_b.values())[-2:] else ""
        highlight_s = " " if apple_vit_s[clf] in sorted(apple_vit_s.values())[-2:] else ""
        print(f"{clf:10} \t{apple_vit_b[clf]:.3f}{highlight_b}\t{apple_vit_s[clf]:.3f}{highlight_s}")

    # Table (e): Classification accuracy of the best shallow classifier
    print("\n(e) Classification accuracy of the best shallow classifier from previous experiments in combination with feature embeddings from both ViTs and CNNs. The first and second best score for each data set are highlighted.")
    print("Data set \t->\tFayoum Banana\tCASC IFW")
    best_banana = max(max(banana_vit_b.values()), max(banana_vit_s.values()), max(banana_clf.values()))
    best_apple = max(max(apple_vit_b.values()), max(apple_vit_s.values()), max(apple_clf.values()))
    highlight_banana = " " if best_banana in sorted([best_banana, best_apple])[-2:] else ""
    highlight_apple = " " if best_apple in sorted([best_banana, best_apple])[-2:] else ""
    print(f"Embeddings->\tDINO ViT-B/8\t{best_banana:.3f}{highlight_banana}\t{best_apple:.3f}{highlight_apple}")
    print(f"DINO ViT-S/8\t{best_banana:.3f}{highlight_banana}\t{best_apple:.3f}{highlight_apple}")
    print(f"CNN\t\t{max(banana_full.values()):.3f}{highlight_banana}\t{max(apple_full.values()):.3f}{highlight_apple}")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 170MB/s]
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 139MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 180MB/s]
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 184MB/s]
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-0676ba

Processing Banana Dataset (Fayoum banana)...
AlexNet features shape: (2485, 64)
DenseNet features shape: (2485, 64)
ResNet-101 features shape: (2485, 64)
ResNet-152 features shape: (2485, 64)
ResNet-18 features shape: (2485, 64)
ResNet-50 features shape: (2485, 64)
SqueezeNet features shape: (2485, 64)
VGG features shape: (2485, 64)

Processing Apple Dataset (CASC IFW apple)...
AlexNet features shape: (2403, 64)
DenseNet features shape: (2403, 64)
ResNet-101 features shape: (2403, 64)
ResNet-152 features shape: (2403, 64)
ResNet-18 features shape: (2403, 64)
ResNet-50 features shape: (2403, 64)
SqueezeNet features shape: (2403, 64)
VGG features shape: (2403, 64)

(a) CNN results for the Fayoum banana data set. We report both accuracies when all weights are trained and those when only the dense classification layers are trained. The first and second best scores for each variation are highlighted.
Model		Full model	Classifier only
AlexNet    0.897 **	0.892 **
DenseNet   0.916 **	0.767 **